## Table Content:

#### 1. Explore the data
#### 2. Fix missing values
#### 3. Fix incorrect values
#### 4. Fix incorrect values ('Country') using FuzzyWuzzy package (Manual corrections for the column 'Country')

In [269]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet
import fuzzywuzzy
from fuzzywuzzy import process
import chardet


data = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses).csv')

data.head(3)

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White


## 1. Explore the data

In [211]:
data.shape

(27538, 18)

In [212]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27538 entries, 0 to 27537
Data columns (total 18 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Timestamp                                 27538 non-null  object 
 1   How old are you?                          27538 non-null  object 
 2   Industry                                  27470 non-null  object 
 3   Job title                                 27538 non-null  object 
 4   Additional context on job title           7131 non-null   object 
 5   Annual salary                             27538 non-null  object 
 6   Other monetary comp                       20416 non-null  float64
 7   Currency                                  27538 non-null  object 
 8   Currency - other                          180 non-null    object 
 9   Additional context on income              3001 non-null   object 
 10  Country                           

### Missing values

In [213]:
data.isnull().sum()

Timestamp                                       0
How old are you?                                0
Industry                                       68
Job title                                       0
Additional context on job title             20407
Annual salary                                   0
Other monetary comp                          7122
Currency                                        0
Currency - other                            27358
Additional context on income                24537
Country                                         0
State                                        4901
City                                           75
Overall years of professional experience        0
Years of experience in field                    0
Highest level of education completed          206
Gender                                        163
Race                                          158
dtype: int64

## 2. Fix missing values

#### columns: 'Industry', 'Highest level of education completed', 'Other monetary comp'

I'm going to replaced NaN values by 0.

In [214]:
data['Industry'] = data['Industry'].fillna(0)
data['Highest level of education completed'] = data['Highest level of education completed'].fillna(0)
data['Other monetary comp'] = data['Other monetary comp'].fillna(0)

 #### columns: 'State', 'City', 'Gender', 'Race'
 
 Replace NaNs with the string 'NoDataAvailable'

In [215]:
data.loc[data['State'].isna(), 'State'] = 'NoDataAvailable'
data.loc[data['City'].isna(), 'City'] = 'NoDataAvailable'
data.loc[data['Gender'].isna(), 'Gender'] = 'NoDataAvailable'
data.loc[data['Race'].isna(), 'Race'] = 'NoDataAvailable'

#### Drop values

The columns 'Additional context on job title', 'Additional context on income' and 'Currency - other' barely have any values, so we gonna drop them.

In [216]:
data = data.drop(['Additional context on job title', 'Additional context on income','Currency - other'], axis=1)

## 3. Fix incorrect values

#### column: 'Gender'

In [217]:
data['Gender'].unique()

array(['Woman', 'Non-binary', 'Man', 'NoDataAvailable',
       'Other or prefer not to answer', 'Prefer not to answer'],
      dtype=object)

In [218]:
data['Gender'].replace({'Other or prefer not to answer': 'Prefer not to answer'}, inplace=True)

In [219]:
data['Gender'].unique()

array(['Woman', 'Non-binary', 'Man', 'NoDataAvailable',
       'Prefer not to answer'], dtype=object)

#### column: 'Timestamp'

In [220]:
data['Timestamp'].dtype

dtype('O')

In [221]:
# convert to datetime type
data['Timestamp'] = pd.to_datetime(data['Timestamp'])


#### column: Years of experience in field

In [222]:
data['Years of experience in field'] = data['Years of experience in field'].str.replace('years', '')

# and the other one
data['Overall years of professional experience'] = data['Overall years of professional experience'].str.replace('years', '')

data['Years of experience in field'].value_counts()

11 - 20           6461
5-7               6412
2 - 4             6102
8 - 10            4895
21 - 30           1851
1 year or less    1403
31 - 40            376
41  or more         38
Name: Years of experience in field, dtype: int64

#### column: 'How old are you?'

In [223]:
# change the column name from 'How old are you?' to 'Age Group'

data.rename(columns={'How old are you?': 'Age Group'}, inplace=True)

## 4. Fix incorrect values ('Country')

There are a lot of similar and random values in the 'Country' column. Lets fix this.

In [224]:
data_countries = data['Country'].unique()
data_countries.sort()
data_countries

array([' New Zealand', ' U.S.', ' US', ' United States',
       '$2,175.84/year is deducted for benefits',
       'ARGENTINA BUT MY ORG IS IN THAILAND', 'Afghanistan', 'Africa',
       'America', 'Aotearoa New Zealand', 'Argentina', 'Australi',
       'Australia', 'Australia ', 'Australian ', 'Austria',
       'Austria, but I work remotely for a Dutch/British company',
       'Bangladesh', 'Bangladesh ', 'Belgium', 'Belgium ', 'Bermuda',
       'Brasil', 'Brazil', 'Brazil ', 'Britain ', 'Bulgaria', 'CANADA',
       'CANADA ', 'California ', 'Cambodia', 'Can', 'Canad', 'Canada',
       'Canada ', 'Canada and USA', 'Canada, Ottawa, ontario', 'Canadw',
       'Canadá', 'Canda', 'Catalonia', 'Cayman Islands', 'Chile', 'China',
       'Colombia', 'Company in Germany. I work from Pakistan.', 'Congo',
       'Contracts', 'Costa Rica', "Cote d'Ivoire", 'Croatia ', 'Csnada',
       'Cuba', 'Currently finance', 'Cyprus', 'Czech Republic',
       'Czech Republic ', 'Czech republic', 'Czechia', 'D

In [225]:
# convert the values to lowercase
data['Country'] = data['Country'].str.lower()

# remove trailing white spaces
data['Country'] = data['Country'].str.strip()

In [226]:
data_countries = data['Country'].unique()
data_countries.sort()
# data_countries

### String Matching with FuzzyWuzzy

I'm gonna use the FuzzyWuzzy library for string matches.

#### Canada

In [227]:
# get the top 10 closest matches to "canada"
matches = process.extract('canada', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('canada', 100),
 ('canad', 91),
 ('canadá', 91),
 ('canda', 91),
 ('canadw', 83),
 ('csnada', 83),
 ('can', 67),
 ('rwanda', 67),
 ('uganda', 67),
 ('canada and usa', 60)]

So I need some function to able to replace matching strings.

In [228]:
# I took this function from Zeeshan Usmani

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [229]:
replace_matches_in_column(df=data, column='Country', string_to_match='canada', min_ratio = 80)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


#### usa

In [230]:
# get the top 10 closest matches to "u.s.a"
matches = process.extract('u.s.a', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('u.s.a', 100),
 ('u.s.a.', 100),
 ('u.s.', 75),
 ('u.s>', 75),
 ('u.s', 75),
 ('u. s.', 75),
 ('u.a.', 75),
 ('u. s', 75),
 ('us of a', 67),
 ('u.sa', 67)]

In [231]:
replace_matches_in_column(df=data, column='Country', string_to_match='u.s.a', min_ratio = 67)

All done!


In [232]:
data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

In [233]:
# get the top 10 closest matches to "usa"
matches = process.extract('usa', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('usa', 100),
 ('usab', 86),
 ('usat', 86),
 ('usaa', 86),
 ('us', 80),
 ('ua', 80),
 ('isa', 67),
 ('russia', 67),
 ('uss', 67),
 ('uae', 67)]

In [234]:
replace_matches_in_column(df=data, column='Country', string_to_match='usa', min_ratio = 66)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


In [235]:
# get the top 10 closest matches to "united states"
matches = process.extract('united states', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('united states', 100),
 ('united  states', 100),
 ('united state', 96),
 ('united sates', 96),
 ('uniited states', 96),
 ('unted states', 96),
 ('united statesp', 96),
 ('united stattes', 96),
 ('united statees', 96),
 ('uniteed states', 96)]

In [236]:
replace_matches_in_column(df=data, column='Country', string_to_match='united states', min_ratio = 96)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


Now I'm gonna do the same thing for all incorrect 'united states' values multiple times, untill we get better results.

In [237]:
replace_matches_in_column(df=data, column='Country', string_to_match='united states', min_ratio = 92)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


In [238]:
# get the top 10 closest matches to "united states"
matches = process.extract('united states', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

#matches

replace_matches_in_column(df=data, column='Country', string_to_match='united states', min_ratio = 68)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


In [239]:
# get the top 10 closest matches to "united states"
matches = process.extract('united states', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('united states', 100),
 ('unitef stated', 85),
 ('united states of america', 70),
 ('united states is america', 70),
 ('united states of american', 68),
 ('united states of americas', 68),
 ('united states- puerto rico', 68),
 ('united state of america', 67),
 ('united sates of america', 67),
 ('united arab emirates', 67)]

In [240]:
replace_matches_in_column(df=data, column='Country', string_to_match='united states of america', min_ratio = 70)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


In [241]:
# get the top 10 closest matches to "united states of america"
matches = process.extract('united states of america', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

matches

[('united states of america', 100),
 ('united arab emirates', 64),
 ('unitef stated', 59),
 ('saudi arabia', 50),
 ('for the united states government, but posted overseas', 47),
 ('america', 45),
 ('south africa', 44),
 ('united kindom', 43),
 ('san francisco', 43),
 ('united kingdom', 42)]

#### united kingdom

In [242]:
# get the top 10 closest matches to "united kingdom"
matches = process.extract('united kingdom', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

#matches

replace_matches_in_column(df=data, column='Country', string_to_match='united kingdom', min_ratio=93)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


#### australia

In [243]:
# get the top 10 closest matches to "australia"
matches = process.extract('australia', data_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) 

#matches

replace_matches_in_column(df=data, column='Country', string_to_match='australia', min_ratio=93)

data_countries = data['Country'].unique()
# data_countries.sort()
# data_countries

All done!


In [244]:
# convert the values to uppercase

data['Country'] = data['Country'].str.title()


In [245]:
# let's delete rows that cointain no data about counrty

no_country_values = data[(data['Country'] == 'N/A (Remote From Wherever I Want)') |
                              (data['Country'] == 'From New Zealand But On Projects Across Apac') |
                              (data['Country'] == 'I Was Brought In On This Salary To Help With The Ehr And Very Quickly Was Promoted To Current Position But Compensation Was Not Altered.') |
                              (data['Country'] == "I Earn Commission On Sales. If I Meet Quota, I'M Guaranteed Another 16K Min. Last Year I Earned An Additional 27K. It'S Not Uncommon For People In My Space To Earn 100K+ After Commission.") |
                              (data['Country'] == '$2,175.84/Year Is Deducted For Benefits') |
                              (data['Country'] == 'Contracts') |
                              (data['Country'] == 'Global') |
                              (data['Country'] == 'International') |
                              (data['Country'] == 'Na') |
                              (data['Country'] == 'Policy') |
                              (data['Country'] == 'United Y') |
                              (data['Country'] == "We Don'T Get Raises, We Get Quarterly Bonuses, But They Periodically Asses Income In The Area You Work, So I Got A Raise Because A 3Rd Party Assessment Showed I Was Paid Too Little For The Area We Were Located") |
                              (data['Country'] == 'Worldwide (Based In Us But Short Term Trips Aroudn The World)') |
                              (data['Country'] == 'Canada And Usa') |
                              (data['Country'] == 'I.S.') |
                              (data['Country'] == 'Is') |
                              (data['Country'] == 'Y') |
                              (data['Country'] == 'united kingdom') |
                              (data['Country'] == 'From Romania, But For An Us Based Company') |
                              (data['Country'] == 'Bonus Based On Meeting Yearly Goals Set W/ My Supervisor')].index

data.drop(no_country_values, inplace = True)

## So Fuzzy package didn't correct 100% so I decided to manually do same changes:

In [255]:
data['Country'] = data['Country'].replace({'U.S.A' : 'USA',
                                            'Uxz': 'USA',
                                            'Virginia': 'USA',
                                            'united states of america': 'USA', 
                                            'unitef stated': 'USA',
                                            'usd': 'USA',
                                            'america': 'USA',
                                            '🇺🇸': 'USA',
                                            'Us Govt Employee Overseas, Country Withheld': 'USA',
                                            'Usa Tomorrow': 'USA',
                                            'The Us': 'USA',
                                            'Usa-- Virgin Islands': 'USA',
                                            'San Francisco': 'USA',
                                            'California': 'USA',
                                            'Us Govt Employee Overseas, Country Withheld': 'USA',
                                            'Usa (Company Is Based In A Us Territory, I Work Remote)': 'USA',
                                            'United States (I Work From Home And My Clients Are All Over The Us/Canada/Pr': 'USA',
                                            'Englang': 'united kingdom',
                                            'United Kingdom (England)': 'united kingdom',
                                            'Uk For U.S. Company': 'united kingdom',   
                                            'u.k.': 'united kingdom', 
                                            'u.k': 'united kingdom',
                                            'Wales': 'united kingdom',
                                            'England': 'united kingdom',
                                            'great britain': 'united kingdom',
                                            'England/Uk': 'united kingdom',
                                            'uk': 'united kingdom',
                                            'Britain': 'united kingdom',
                                            'Wales (United Kingdom)': 'united kingdom',
                                            'Uk, Remote': 'united kingdom',
                                            'Company In Germany. I Work From Pakistan.': 'Germany',
                                            'I Work For A Uae-Based Organization, Though I Am Personally In The Us.': 'UAE',
                                            'United Arab Emirates': 'UAE',
                                            'Uk, But For Globally Fully Remote Company': 'united kingdom',
                                            'Remote (Philippines)': 'Philippines',
                                            'Italy (South)': 'Italy',
                                            'Can': 'Canada',
                                            'Canada, Ottawa, Ontario': 'Canada',
                                            'I Am Located In Canada But I Work For A Company In The Us': 'Canada',
                                            'Japan, Us Gov Position': 'Japan',
                                            'Nederland': 'The Netherlands',
                                            'Nl': 'The Netherlands',
                                            'Hong Konh': 'Hong Kong',
                                            'United States Of America': 'USA',
                                            'United Kingdom': 'UK',
                                            'united kingdom': 'UK',
                                            'England, Uk.': 'UK',
                                            'America' : 'USA',
                                            'U.K.': 'UK',
                                            'Uk': 'UK',
                                            'U.K': 'UK',
                                            'Wales, Uk': 'UK',
                                            'Wales (Uk)':'UK',
                                            'Currently Finance': 'France',
                                            'New Zealand Aotearoa': 'New Zealand ',
                                            'Netherlands': 'The Netherlands',
                                            'Uk (Northern Ireland)': 'UK',
                                            'Nz': 'New Zealand',
                                            'For The United States Government, But Posted Overseas': 'USA',
                                            'England, United Kingdom': 'UK',
                                            'Uk (England)': 'UK',
                                            'Northern Ireland, United Kingdom' : 'UK',
                                            'Northern Ireland': 'UK',
                                            'England, Uk': 'UK',
                                            'Scotland, Uk': 'Scotland',
                                            'Austria, But I Work Remotely For A Dutch/British Company': 'Austria',
                                            'Unitef Stated': 'USA',
                                            'Usd': 'USA',
                                            'Aotearoa New Zealand': 'New Zealand',
                                            "Usa, But For Foreign Gov'T": 'USA',
                                            'England, Gb': 'UK',
                                            'Unitedstates': 'USA',
                                            'U.K. (Northern England)': 'UK',
                                            'New Zealand  ': 'New Zealand ',
                                            'Danmark': 'Denmark',
                                            'Chile': 'Chili',
                                            'Great Britain': 'UK',
                                            'Hartford': 'USA',
                                            'Isle Of Man': 'UK',
                                             'Usa': 'USA',
                                          "I Work For An Us Based Company But I'M From Argentina.": 'Argentina',
                                           'Argentina But My Org Is In Thailand': 'Argentina',
                                           'united kingdom': 'UK'
                                          })

In [256]:
# let's check
data['Country'].unique()

array(['USA', 'UK', 'Canada', 'Scotland', 'The Netherlands', 'Australia',
       'Spain', 'Finland', 'France', 'Germany', 'Ireland', 'India',
       'Argentina', 'Denmark', 'Switzerland', 'Bermuda', 'Malaysia',
       'Mexico', 'South Africa', 'Belgium', 'Sweden', 'Hong Kong',
       'Kuwait', 'Norway', 'Sri Lanka', 'Greece', 'Japan', 'Austria',
       'Brazil', 'Hungary', 'Luxembourg', 'Colombia', 'New Zealand',
       'Trinidad And Tobago', 'Cayman Islands', 'Czech Republic',
       'Czechia', 'Latvia', 'Puerto Rico', 'Rwanda', 'UAE', 'Bangladesh',
       'Romania', 'Serbia', 'Philippines', 'Poland', 'Turkey',
       'Catalonia', 'Italy', 'Jersey, Channel Islands', 'China',
       'Afghanistan', 'Israel', 'New Zealand ', 'Mainland China',
       'Taiwan', 'Cambodia', 'Vietnam', 'Remote', 'Singapore',
       'South Korea', 'Thailand', 'Lithuania', 'Eritrea', 'Indonesia',
       'Cuba', 'Slovenia', "Cote D'Ivoire", 'Somalia', 'Slovakia',
       'Portugal', 'Sierra Leone', 'The Bahamas'

#### column: Annual salary

In [257]:
data['Annual salary'].dtype

dtype('O')

In [258]:
# let's convert to numeric value
data['Annual salary'] = data['Annual salary'].str.replace(',', '')
data['Annual salary'] = data['Annual salary'].astype(float)


In [259]:
data['Currency'].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'Other', 'CHF', 'ZAR',
       'SEK', 'HKD', 'JPY'], dtype=object)

We have 10 different currencies and 1 'Other'. <br>
It'll be better if we convert all the currencies into 1 currency (in this case to USD), but before that I'll drop the 'Other' values.

In [260]:
data.drop(data[data['Currency']=='Other'].index, inplace=True)

I searched for all currencies's exchange rates and now I'm going to create a DataFrame with those currencies:

In [261]:
rates = {'Currency': ['USD', 'GBP', 'CAD','EUR', 'AUD/NZD', 'CHF', 'ZAR', 'SEK', 'HKD', 'JPY'],
        'Rate': [1,1.332195,0.790515, 1.120805, 0.953825178, 1.068555, 0.062795,0.110174, 0.12826, 0.008669]}

data_rates = pd.DataFrame(rates)

# join togeher two dataframes
new_data = pd.merge(data, data_rates, how='left')

# add a new column - Salary (USD)
new_data['Salary (USD)'] = new_data['Annual salary']*new_data['Rate']
new_data['Salary (USD)'] = new_data['Salary (USD)'].astype(int)

## Clean dataset, ready for data visualization and data analysis.

In [265]:
clear_data = new_data.to_csv('AskaManager_clean.csv', index_label=False)

# cleared_data.to_parquet('cleared_data.parquet.gzip', compression='gzip')